In [1]:
import cv2
import numpy as np
from skimage.util import random_noise
from tqdm.notebook import tqdm
from skimage.transform import rotate
import os

In [3]:
def make_paper(image, num):
    paper = cv2.imread("paper_textures/" + str(num) + ".webp")
    paper = cv2.resize(paper, (image.shape[1], image.shape[0]))
    
    mask = (image[:, :, 0] == 255) * (image[:, :, 1] == 255) * (image[:, :, 2] == 255)
    mask = np.stack([mask, mask, mask]).transpose((1, 2, 0))

    return paper * mask + image * (1 - mask)

def make_background(image, num):
    background = cv2.imread("surface_textures/" + str(num) + ".jpeg")
    
    image_long_side = np.random.randint(background.shape[0] * 0.2, background.shape[0] * 0.5)
    image = cv2.resize(image, (int(image.shape[1] / image.shape[0] * image_long_side), image_long_side))
    
    h_pad = np.random.randint(0, background.shape[0] - image.shape[0])
    w_pad = np.random.randint(0, background.shape[1] - image.shape[1])
    image = np.pad(image, ((h_pad, background.shape[0] - image.shape[0] - h_pad), 
                           (w_pad, background.shape[1] - image.shape[1] - w_pad), 
                           (0, 0)))
    image = image * 255
    
    mask = (image[:, :, 0] == 0) * (image[:, :, 1] == 0) * (image[:, :, 2] == 0)
    mask = np.stack([mask, mask, mask]).transpose((1, 2, 0))

    return background * mask + image * (1 - mask)

In [5]:
def print_defect(img):
    shifted_rows = np.random.binomial(1, 0.01, img.shape[0])
    for i in range(img.shape[0]):
        if shifted_rows[i] == 1:
            shift = np.random.randint(-25, 25)
            new_row = np.zeros_like(img[i])
            if shift > 0:
                new_row[shift:] = img[i][:-shift]
            elif shift < 0:
                new_row[:shift] = img[i][-shift:]
            else:
                new_row = img[i]
            img[i] = new_row
    return img

def blur(img, shape = 5):
    kern = np.random.randint(1, shape)
    return cv2.blur(img, (kern, kern))

In [114]:
def aug(image, num):
    image = make_paper(image, 11 if num == 9 else num % 5 + 11) / 255
    image = rotate(image, np.random.uniform(0, 360), resize=True)
    image = print_defect(image)
    image = make_background(image, num + 1)
    image = blur(image)
    return image

In [124]:
for file in tqdm(os.listdir('images//')):
    image = cv2.imread('images/' + file)
    for i in range(10):
        cv2.imwrite('res/' + file[:-4]+ '_' + str(i) + '.png', aug(image, i))

  0%|          | 0/10 [00:00<?, ?it/s]